In [1]:
# if hit plotting library issues, try resetting LD path for julia
# can set in ~/.local/share/jupyter/kernels/
haskey(ENV, "LD_LIBRARY_PATH") && @assert ENV["LD_LIBRARY_PATH"] == ""
import Pkg
pkgs = [
    "Revise",
    "FASTX",
    "BioSequences",
    "Kmers",
    "Graphs",
    "MetaGraphs",
    "SparseArrays",
    "ProgressMeter",
    "Distributions",
    "HiddenMarkovModels",
    "BioAlignments",
    "StatsBase",
    "Random",
    "StatsPlots",
    "Statistics",
    # "GraphMakie",
    "IterTools",
    "Primes",
    "OnlineStats",
    "IteratorSampling",
    "HypothesisTests",
    "Clustering",
    "Distances",
    "BioAlignments",
    "Statistics",
    "Primes",
    "DataFrames"
]
# Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end
# Pkg.develop(path="/global/cfs/projectdirs/m4269/cjprybol/Mycelia")
# Pkg.develop(path="../../..")
import Mycelia

┌ Warning: The active manifest file is an older format with no julia version entry. Dependencies may have been resolved with a different julia version.
└ @ /global/cfs/cdirs/m4269/cjprybol/Mycelia/Manifest.toml:0
[ Info: Precompiling Mycelia [453d265d-8292-4a7b-a57c-dce3f9ae6acd]
[ Info: Skipping precompilation since __precompile__(false). Importing Mycelia [453d265d-8292-4a7b-a57c-dce3f9ae6acd].


In [2]:
PROJECT_BASEDIR = dirname(pwd())
data_dir = joinpath(PROJECT_BASEDIR, "data")
genome_dir = mkpath(joinpath(data_dir, "genomes"))

"/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/variant-calling-benchmarking/data/genomes"

In [3]:
working_dir = joinpath(data_dir, "test")
mkpath(working_dir)

"/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/variant-calling-benchmarking/data/test"

In [4]:
# short_read_sets = unique(map(x -> match(r"^(.+\.\d+x)\.", x).captures[1], filter(x -> occursin(r"\.fna\.art", x) && occursin(r"\.fq\.gz", x) && !occursin("trimming_report", x) && !occursin("_val_", x), sort(readdir(genome_dir, join=true), by=x->filesize(x)))))
# # forward = short_read_set * ".1_val_1.fq.gz"
# # reverse = short_read_set * ".2_val_2.fq.gz"

In [5]:
long_read_fastqs = sort(filter(x -> occursin(r"\.filtlong\.fq\.gz$", x), readdir(genome_dir, join=true)), by=x->filesize(x))
# fastq = long_read_fastqs[1]
# fastq = long_read_fastqs[2]
# fastq = long_read_fastqs[3]
original_fastq = long_read_fastqs[4]

"/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/variant-calling-benchmarking/data/genomes/3NEzu1DmBy.fna.normalized.vcf.fna.badread.10x.filtlong.fq.gz"

In [6]:
# # 1 hour runtime
# polishing_results = Mycelia.iterative_polishing(original_fastq)

In [7]:
# assembly_fastq = last(polishing_results).fastq

In [8]:
# assembly_k = last(polishing_results).k

In [9]:
assembly_fastq = "/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/variant-calling-benchmarking/data/genomes/3NEzu1DmBy.fna.normalized.vcf.fna.badread.10x.filtlong.k11.k13.k17.k19.k23.k31.k53.k89.fq.gz"
assembly_k = 89

89

In [10]:
mean_read_length = Statistics.mean([length(FASTX.sequence(record)) for record in Mycelia.open_fastx(assembly_fastq)])

8487.583333333334

In [11]:
# assembly_k = maximum(filter(k -> k < mean_read_length/2, Mycelia.ks()))

In [ ]:
# assembly_ks_to_try = filter(k -> k < mean_read_length/2, Mycelia.ks())
# assembly_ks_to_try = filter(k -> k >= assembly_k, assembly_ks_to_try)

In [12]:
# for k in assembly_ks_to_try
#     kmer_graph = Mycelia.build_directed_kmer_graph(fastq=assembly_fastq, k=assembly_k)
#     initial_connected_components = length(Graphs.connected_components(kmer_graph))
#     @show k, initial_connected_components
# end

In [13]:
# score: 47549
# assembly_k = 89
# score: 47549
# assembly_k = 139
# score: 47549
# assembly_k = 233
# score: 47549
assembly_k = 379
# assembly_k = 607
# assembly_k = 983
# assembly_k = 1597
# assembly_k = 2579
# assembly_k = 4177

379

In [14]:
kmer_graph = Mycelia.build_directed_kmer_graph(fastq=assembly_fastq, k=assembly_k)

{23944, 23954} directed Int64 metagraph with Float64 weights defined by :weight (default weight 1.0)

In [15]:
# heuristic - should be based on something better or removed altogether
max_filter = 5

initial_connected_components = length(Graphs.connected_components(kmer_graph))
coverage_threshold = 1
tresholded_vertices = [i for (i, (kmer, count)) in enumerate(MetaGraphs.get_prop(kmer_graph, :kmer_counts)) if count > coverage_threshold]
candidate_subgraph, vertex_map = Graphs.induced_subgraph(kmer_graph, tresholded_vertices)
while (length(Graphs.connected_components(candidate_subgraph)) == initial_connected_components) && (coverage_threshold < max_filter)
    coverage_threshold += 1
    tresholded_vertices = [i for (i, (kmer, count)) in enumerate(MetaGraphs.get_prop(kmer_graph, :kmer_counts)) if count > coverage_threshold]
    candidate_subgraph, vertex_map = Graphs.induced_subgraph(kmer_graph, tresholded_vertices)
end
coverage_threshold -= 1
@show coverage_threshold

tresholded_vertices = [i for (i, (kmer, count)) in enumerate(MetaGraphs.get_prop(kmer_graph, :kmer_counts)) if count > coverage_threshold]
filtered_graph, filtered_graph_vertex_map = Graphs.induced_subgraph(kmer_graph, tresholded_vertices)
# candidate_subgraph, vertex_map = Graphs.induced_subgraph(kmer_graph, [v for v in Graphs.vertices(kmer_graph) if kmer_graph.vprops[v][:count] >= coverage_threshold])

coverage_threshold = 2


({18772, 18770} directed Int64 metagraph with Float64 weights defined by :weight (default weight 1.0), [1, 3, 4, 5, 6, 7, 9, 10, 11, 12  …  23934, 23935, 23936, 23937, 23938, 23939, 23941, 23942, 23943, 23944])

In [16]:
connected_components = sort(Graphs.connected_components(filtered_graph), by=x->length(x), rev=true)

2-element Vector{Vector{Int64}}:
 [4, 5, 7, 8, 12, 13, 14, 17, 18, 20  …  18755, 18756, 18761, 18762, 18766, 18767, 18768, 18769, 18770, 18772]
 [1, 2, 3, 6, 9, 10, 11, 15, 16, 19  …  18748, 18751, 18757, 18758, 18759, 18760, 18763, 18764, 18765, 18771]

In [17]:
length.(connected_components)

2-element Vector{Int64}:
 9458
 9314

In [18]:
connected_component_sequences = BioSequences.LongDNA{2}[]
for connected_component in connected_components
    connected_component_subgraph, connected_component_subgraph_vertex_map = Graphs.induced_subgraph(filtered_graph, connected_component)
    sorted_connected_component_subgraph = Graphs.topological_sort_by_dfs(connected_component_subgraph)
    dsp_result = Graphs.dijkstra_shortest_paths(connected_component_subgraph, first(sorted_connected_component_subgraph))
    maxdist, maxdist_index = findmax(map(d -> d == Inf ? zero(d) : d, dsp_result.dists))
    connected_component_subgraph_longest_path = Graphs.enumerate_paths(dsp_result, maxdist_index)
    # @show connected_component_subgraph_longest_path
    original_graph_kmer_indices = filtered_graph_vertex_map[connected_component_subgraph_vertex_map[connected_component_subgraph_longest_path]]
    connected_component_subgraph_longest_path_reconstructed_sequence = Mycelia.kmer_path_to_sequence(kmer_graph.gprops[:ordered_kmers][original_graph_kmer_indices])
    push!(connected_component_sequences, connected_component_subgraph_longest_path_reconstructed_sequence)
end
connected_component_sequences
length.(connected_component_sequences)

2-element Vector{Int64}:
 9740
 9692

In [19]:
first_sequence_canonical_kmers = unique(last.(collect(Kmers.EveryCanonicalKmer{Kmers.DNAKmer{assembly_k}}(connected_component_sequences[1]))))
second_sequence_canonical_kmers = unique(last.(collect(Kmers.EveryCanonicalKmer{Kmers.DNAKmer{assembly_k}}(connected_component_sequences[2]))))
intersect(first_sequence_canonical_kmers, second_sequence_canonical_kmers)

1572-element Vector{Kmers.DNAKmer{379, 12}}:
 AAGTTGCAGAGGGGGAGCGTGCTACCTCGGGGAATCACT…AGTCGACGTTGAGAAACGCCGAAAGGCACTCTTTTAACA
 AGTTGCAGAGGGGGAGCGTGCTACCTCGGGGAATCACTG…GTCGACGTTGAGAAACGCCGAAAGGCACTCTTTTAACAT
 CATGTTAAAAGAGTGCCTTTCGGCGTTTCTCAACGTCGA…ACAGTGATTCCCCGAGGTAGCACGCTCCCCCTCTGCAAC
 CCATGTTAAAAGAGTGCCTTTCGGCGTTTCTCAACGTCG…GACAGTGATTCCCCGAGGTAGCACGCTCCCCCTCTGCAA
 CCCATGTTAAAAGAGTGCCTTTCGGCGTTTCTCAACGTC…GGACAGTGATTCCCCGAGGTAGCACGCTCCCCCTCTGCA
 GCAGAGGGGGAGCGTGCTACCTCGGGGAATCACTGTCCA…ACGTTGAGAAACGCCGAAAGGCACTCTTTTAACATGGGA
 CAGAGGGGGAGCGTGCTACCTCGGGGAATCACTGTCCAC…CGTTGAGAAACGCCGAAAGGCACTCTTTTAACATGGGAC
 AGAGGGGGAGCGTGCTACCTCGGGGAATCACTGTCCACG…GTTGAGAAACGCCGAAAGGCACTCTTTTAACATGGGACC
 CGGTCCCATGTTAAAAGAGTGCCTTTCGGCGTTTCTCAA…CCGTGGACAGTGATTCCCCGAGGTAGCACGCTCCCCCTC
 AGGGGGAGCGTGCTACCTCGGGGAATCACTGTCCACGGT…TGAGAAACGCCGAAAGGCACTCTTTTAACATGGGACCGC
 CGCGGTCCCATGTTAAAAGAGTGCCTTTCGGCGTTTCTC…CACCGTGGACAGTGATTCCCCGAGGTAGCACGCTCCCCC
 CCGCGGTCCCATGTTAAAAGAGTGCCTTTCGGCGTTTCT…TCACCGTGGACAGTGATTCCCCG

In [20]:
import BioAlignments
scoremodel = BioAlignments.AffineGapScoreModel(BioAlignments.EDNAFULL, gap_open=-5, gap_extend=-1)
forward_alignment = BioAlignments.pairalign(BioAlignments.OverlapAlignment(), connected_component_sequences[1], connected_component_sequences[2], scoremodel)
reverse_complement_alignment = BioAlignments.pairalign(BioAlignments.OverlapAlignment(), connected_component_sequences[1], BioSequences.reverse_complement(connected_component_sequences[2]), scoremodel)
if BioAlignments.score(forward_alignment) > BioAlignments.score(reverse_complement_alignment)
    best_alignment = forward_alignment
else
    best_alignment = reverse_complement_alignment
end
best_alignment

BioAlignments.PairwiseAlignmentResult{Int64, BioSequences.LongSequence{BioSequences.DNAAlphabet{2}}, BioSequences.LongSequence{BioSequences.DNAAlphabet{2}}}:
  score: 47549
  seq:    0 ----CGGATGTCACAAACTCGCTAAATTACACATACTAAGCACTTACTGGACAAACTGAT   56
                ||||||||||||||||||||||||||||||||||||||||||||||||||||||||
  ref:    1 AACTCGGATGTCACAAACTCGCTAAATTACACATACTAAGCACTTACTGGACAAACTGAT   60

  seq:   57 CTATAGCATTCACCGCGTTAGGCGCCTTTGGGAGCTATCCGAGTGCTTGCAAGGCCGTTC  116
            ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
  ref:   61 CTATAGCATTCACCGCGTTAGGCGCCTTTGGGAGCTATCCGAGTGCTTGCAAGGCCGTTC  120

  seq:  117 TGCCGTTCGGCCCTAGGTTTCCTCCAACCGCTGCCGTCTAGGCTCCAAACGTCATCTACC  176
            ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
  ref:  121 TGCCGTTCGGCCCTAGGTTTCCTCCAACCGCTGCCGTCTAGGCTCCAAACGTCATCTACC  180

  seq:  177 CATAAACGGGTCTAATCGTTTCTACTCGATACATTTATACAGGCGTGCGGCAGTACGCCC  236
            |||||||||||||||||||||||||||||||||||||||||||||||

In [21]:
candidate_assemblies = [connected_component_sequences[1], BioSequences.reverse_complement(connected_component_sequences[2])]

2-element Vector{BioSequences.LongSequence{BioSequences.DNAAlphabet{2}}}:
 CGGATGTCACAAACTCGCTAAATTACACATACTAAGCAC…GGCGACCGATCGAGCAAGACTCGTAGGCGAACTAGCTCT
 AACTCGGATGTCACAAACTCGCTAAATTACACATACTAA…ACGGCGACCGATCGAGCAAGACTCGTAGGCGAACTAGCT

In [23]:
# candidate_1_canonical_kmer_counts = Mycelia.count_canonical_kmers(Kmers.DNAKmer{original_k}, first(candidate_assemblies))
# candidate_2_canonical_kmer_counts = Mycelia.count_canonical_kmers(Kmers.DNAKmer{original_k}, last(candidate_assemblies))
# raw_data_canonical_kmer_counts = Mycelia.count_canonical_kmers(Kmers.DNAKmer{original_k}, original_fastq)

In [ ]:
# candidate_1_canonical_kmer_counts


In [ ]:
# raw_data_canonical_kmer_counts

In [ ]:
# sorted_shared_keys = sort(collect(union(keys(candidate_1_canonical_kmer_counts), keys(raw_data_canonical_kmer_counts))))

In [47]:
function assess_assembly_quality(;assembled_sequence::BioSequences.LongDNA{2}, fastq::String, ks::Vector{Int}=filter(x -> 11 <= x <= 53, Mycelia.ks()))
    results = DataFrames.DataFrame()
    ProgressMeter.@showprogress for k in ks
        observed_canonical_kmer_counts = Mycelia.count_canonical_kmers(Kmers.DNAKmer{k}, fastq)
        assembled_canonical_kmer_counts = Mycelia.count_canonical_kmers(Kmers.DNAKmer{k}, assembled_sequence)
        cosine_distance = kmer_counts_to_cosine_similarity(observed_canonical_kmer_counts, assembled_canonical_kmer_counts)
        js_divergence = kmer_counts_to_js_divergence(observed_canonical_kmer_counts, assembled_canonical_kmer_counts)
        qv = kmer_counts_to_merqury_qv(raw_data_counts=observed_canonical_kmer_counts, assembly_counts=assembled_canonical_kmer_counts)
        push!(results, (;k, cosine_distance, js_divergence, qv))
    end
    return results
end

# function assess_assembly_quality(;assembled_sequence::BioSequences.LongDNA{2}, fastq::String, k::Int)
#     assess_assembly_quality(assembled_sequence=assembled_sequence, fastq=fastq, ks=[k])
# end

function kmer_counts_to_cosine_similarity(kmer_counts_1, kmer_counts_2)
    sorted_shared_keys = sort(collect(union(keys(kmer_counts_1), keys(kmer_counts_2))))
    a = [get(kmer_counts_1, kmer, 0) for kmer in sorted_shared_keys]
    b = [get(kmer_counts_1, kmer, 0) for kmer in sorted_shared_keys]
    # Distances.cosine_dist(a, b) == Distances.cosine_dist(b, a) == Distances.cosine_dist(a ./ sum(a), b ./ sum(b)) == Distances.cosine_dist(b ./ sum(b), a ./ sum(a))
    return Distances.cosine_dist(a, b)
end

function kmer_counts_to_js_divergence(kmer_counts_1, kmer_counts_2)
    sorted_shared_keys = sort(collect(union(keys(kmer_counts_1), keys(kmer_counts_2))))
    a = [get(kmer_counts_1, kmer, 0) for kmer in sorted_shared_keys]
    b = [get(kmer_counts_1, kmer, 0) for kmer in sorted_shared_keys]
    a_norm = a ./ sum(a)
    b_norm = b ./ sum(b)
    # Distances.js_divergence(a ./ sum(a), b ./ sum(b)) == Distances.js_divergence(b ./ sum(b), a ./ sum(a))
    # Distances.js_divergence(a, b) != Distances.js_divergence(a ./ sum(a), b ./ sum(b))
    return Distances.js_divergence(a_norm, b_norm)
end

# function jaccard(set1, set2)
#     union_size = length(union(set1, set2))
#     if union_size == 0
#         return 0.0
#     else
#         return 1.0 - (length(intersect(set1, set2)) / union_size)
#     end
# end

# function kmer_counts_to_jaccard(kmer_counts_1::AbstractDict{Kmers.DNAKmer{k}, Int64}, kmer_counts_2::AbstractDict{Kmers.DNAKmer{k}, Int64}) where k
#     # sorted_shared_keys = sort(collect(union(keys(kmer_counts_1), keys(kmer_counts_2))))
#     # a = [get(kmer_counts_1, kmer, 0) for kmer in sorted_shared_keys]
#     # b = [get(kmer_counts_1, kmer, 0) for kmer in sorted_shared_keys]
#     # a_indices = findall(a .> 0)
#     # b_indices = findall(b .> 0)
#     # return Distances.jaccard(a_indices, b_indices)
#     return jaccard(collect(keys(kmer_counts_1)), collect(keys(kmer_counts_2)))
# end

function kmer_counts_to_merqury_qv(;raw_data_counts::AbstractDict{Kmers.DNAKmer{k,N}, Int}, assembly_counts::AbstractDict{Kmers.DNAKmer{k,N}, Int}) where {k,N}
    # Ktotal = # of kmers found in assembly
    Ktotal = length(keys(assembly_counts))
    # Kshared = # of shared kmers between assembly and readset
    Kshared = length(intersect(keys(raw_data_counts), keys(assembly_counts)))
    # probabilitiy_base_in_assembly_correct
    P = (Kshared/Ktotal)^(1/k)
    # # Error rate
    E = 1-P
    QV = -10log10(E)
    # return (;P, E, QV)
    return QV
end

kmer_counts_to_merqury_qv (generic function with 1 method)

In [48]:
assess_assembly_quality(assembled_sequence=connected_component_sequences[1], fastq=original_fastq)

Progress: 100%|█████████████████████████████████████████| Time: 0:00:02


Row,k,cosine_distance,js_divergence,qv
,Int64,Float64,Float64,Float64
1,11,0.0,0.0,Inf
2,13,1.11022e-16,0.0,43.2347
3,17,0.0,0.0,37.4054
4,19,0.0,0.0,35.5786
5,23,0.0,0.0,32.2259
6,31,1.11022e-16,0.0,28.2846
7,53,0.0,0.0,23.2896


In [49]:
assess_assembly_quality(assembled_sequence=connected_component_sequences[2], fastq=original_fastq)

Progress: 100%|█████████████████████████████████████████| Time: 0:00:00


Row,k,cosine_distance,js_divergence,qv
,Int64,Float64,Float64,Float64
1,11,0.0,0.0,Inf
2,13,1.11022e-16,0.0,50.9966
3,17,0.0,0.0,40.3974
4,19,0.0,0.0,38.3242
5,23,0.0,0.0,34.0077
6,31,1.11022e-16,0.0,28.8476
7,53,0.0,0.0,23.4312


In [ ]:
# 1 calculate most likely of the two paths from the original data

In [ ]:
# 2 calculate most likely of the two paths from the final fastq

In [ ]:
# 3 report out both alleles

In [52]:
# take the second - it's higher QV
final_assembly_sequences = [connected_component_sequences[2]]
assembly_fasta = replace(assembly_fastq, Mycelia.FASTQ_REGEX => ".assembly_k$(assembly_k).fna")
open(assembly_fasta, "w") do io
    fastx_io = FASTX.FASTA.Writer(io)
    for (i, sequence) in enumerate(final_assembly_sequences)
        identifier = "contig$(i)_length=$(length(sequence))"
        fasta_record = FASTX.FASTA.Record(identifier, sequence)
        write(fastx_io, fasta_record)
    end
    close(fastx_io)
end
assembly_fasta

"/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/variant-calling-benchmarking/data/genomes/3NEzu1DmBy.fna.normalized.vcf.fna.badread.10x.filtlong.k11.k13.k17.k19.k23.k31.k53.k89.assembly_k379.fna"

In [ ]:
# readlines(assembly_fasta)